In [1]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

src_path = os.path.join(project_root, 'src')
sys.path.insert(0, src_path)  
os.chdir(project_root)

print("src_path added to sys.path:", src_path in sys.path)

src_path added to sys.path: True


In [2]:
from src.dq_plus import AmericanOptionPricing, QuadratureType
from src.Option import OptionType, EuropeanOption

import pandas as pd
import time

In [3]:
K = 100.       # Strike price
S = 100.
T = 1.0         # Time to maturity (in years)
r = 0.05        # Risk-free interest rate
q = 0.05        # Dividend yield
sigma = 0.25    # Volatility
tau_max = 1.
eta = 0.5

l,m,n = 201, 16, 64
p=201

In [5]:
EuropeanOption.european_put_value(tau_max, K, r, q, sigma, K)

9.462492596167081

In [9]:
l_values = [5, 7, 11, 15, 15, 25, 25, 25, 35, 51, 65]
m_values = [1, 2, 2, 2, 3, 4, 5, 6, 8, 8, 8]
n_values = [4, 5, 5, 6, 7, 9, 12, 15, 16, 24, 32]
p_values = [15, 20, 31, 41, 41, 51, 61, 61, 81, 101, 101]

results = []

In [11]:
# benchmark
benchmark = AmericanOptionPricing(
        K=K,
        r=r,
        q=q,
        vol=sigma,
        tau_max=tau_max,
        n=n,
        l=l,
        p=p,
        m=m,
        option_type=OptionType.Put,
        quadrature_type=QuadratureType.tanh_sinh,
        eta=0.5
    )
benchmark.run_full_algorithm()
benchmark.compute_pricing_points()
benchmark.compute_option_pricing(S=100)
benchmark_premium = benchmark.compute_option_pricing(S=100)[1]

benchmark_premium

Starting iteration 1/16
Iteration 1/16 completed.
Starting iteration 2/16
Iteration 2/16 completed.
Starting iteration 3/16
Iteration 3/16 completed.
Starting iteration 4/16
Iteration 4/16 completed.
Starting iteration 5/16
Iteration 5/16 completed.
Starting iteration 6/16
Iteration 6/16 completed.
Starting iteration 7/16
Iteration 7/16 completed.
Starting iteration 8/16
Iteration 8/16 completed.
Starting iteration 9/16
Iteration 9/16 completed.
Starting iteration 10/16
Iteration 10/16 completed.
Starting iteration 11/16
Iteration 11/16 completed.
Starting iteration 12/16
Iteration 12/16 completed.
Starting iteration 13/16
Iteration 13/16 completed.
Starting iteration 14/16
Iteration 14/16 completed.
Starting iteration 15/16
Iteration 15/16 completed.
Starting iteration 16/16
Iteration 16/16 completed.
Jacobi-Newton iterations completed.


0.10695282147080121

In [12]:
for l, m, n, p in zip(l_values, m_values, n_values, p_values):
    if l == 5 or l == 7:
        solver = AmericanOptionPricing(
            K=K,
            r=r,
            q=q,
            vol=sigma,
            tau_max=tau_max,
            n=n,
            l=l,
            p=p,
            m=m,
            option_type=OptionType.Put,
            quadrature_type=QuadratureType.Gauss_Legendre,
            eta=eta
        )
    else:
        solver = AmericanOptionPricing(
            K=K,
            r=r,
            q=q,
            vol=sigma,
            tau_max=tau_max,
            n=n,
            l=l,
            p=p,
            m=m,
            option_type=OptionType.Put,
            quadrature_type=QuadratureType.tanh_sinh,
            eta=eta
        )

    start_time = time.time()

    solver.run_full_algorithm()
    solver.compute_pricing_points()
    solver.compute_option_pricing(S=S)
    premium = solver.compute_option_pricing(S=S)[1]

    end_time = time.time()

    cpu_seconds = end_time - start_time

    relative_error = abs(premium - benchmark_premium) / benchmark_premium
    results.append([f"({l},{m},{n})", p, premium, relative_error, cpu_seconds])

df = pd.DataFrame(results, columns=["(l,m,n)", "p", "American Premium", "Relative Error", "CPU Seconds"])


Starting iteration 1/1
Iteration 1/1 completed.
Jacobi-Newton iterations completed.
Starting iteration 1/2
Iteration 1/2 completed.
Starting iteration 2/2
Iteration 2/2 completed.
Jacobi-Newton iterations completed.
Starting iteration 1/2
Iteration 1/2 completed.
Starting iteration 2/2
Iteration 2/2 completed.
Jacobi-Newton iterations completed.
Starting iteration 1/2
Iteration 1/2 completed.
Starting iteration 2/2
Iteration 2/2 completed.
Jacobi-Newton iterations completed.
Starting iteration 1/3
Iteration 1/3 completed.
Starting iteration 2/3
Iteration 2/3 completed.
Starting iteration 3/3
Iteration 3/3 completed.
Jacobi-Newton iterations completed.
Starting iteration 1/4
Iteration 1/4 completed.
Starting iteration 2/4
Iteration 2/4 completed.
Starting iteration 3/4
Iteration 3/4 completed.
Starting iteration 4/4
Iteration 4/4 completed.
Jacobi-Newton iterations completed.
Starting iteration 1/5
Iteration 1/5 completed.
Starting iteration 2/5
Iteration 2/5 completed.
Starting iterati

In [13]:
df

,"(l,m,n)",p,American Premium,Relative Error,CPU Seconds
0,"(5,1,4)",15,0.109893,0.027492,0.028666
1,"(7,2,5)",20,0.108360,0.013159,0.039296
2,"(11,2,5)",31,0.108361,0.013167,0.038025
3,"(15,2,6)",41,0.108376,0.013311,0.049134
4,"(15,3,7)",41,0.107673,0.006738,0.072467
5,"(25,4,9)",51,0.107321,0.003442,0.156964
6,"(25,5,12)",61,0.107142,0.001766,0.239185
7,"(25,6,15)",61,0.107050,0.000908,0.342008
8,"(35,8,16)",81,0.106978,0.000239,0.627159
9,"(51,8,24)",101,0.106978,0.000239,1.308139


In [ ]:
df